# [#base_data] - Analyse et traitement des données sources

## 1 - Import des modules

In [1]:
import pandas as pd

## 2 - Import des données

In [2]:
# Import des données
spreadsheet_id = "1EQ3eFvU0fvaok_Ifl0sijaS1_WD_5xycf8kItBilmxw" 
sheet_id = "480090483"
url_data = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={sheet_id}"

In [3]:
# Conversion en DataFrame et GeoDataFrame
data = pd.read_csv(url_data)

## 3 - Exploration et nettoyage des données 

### 3.1 - Structure de données

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dep-code               101 non-null    object 
 1   dep-code-txt           101 non-null    object 
 2   dep-nom                101 non-null    object 
 3   sapin-nb-exploitation  84 non-null     float64
 4   sapin-nb-hectare       82 non-null     float64
 5   total-nb-expl          101 non-null    int64  
 6   total-nb-ha            99 non-null     float64
 7   pct-expl               101 non-null    object 
 8   pct-hect               101 non-null    object 
 9   region                 8 non-null      object 
dtypes: float64(3), int64(1), object(6)
memory usage: 8.0+ KB


In [5]:
data.head()

,dep-code,dep-code-txt,dep-nom,sapin-nb-exploitation,sapin-nb-hectare,total-nb-expl,total-nb-ha,pct-expl,pct-hect,region
0,58,58,Nièvre,59.0,879.0,2715,367818.0,"2,2","0,2",morvan
1,29,29,Finistère,47.0,650.0,5978,382582.0,"0,8","0,2",bretagne
2,21,21,Côte-d'Or,39.0,378.0,4093,462011.0,"1,0","0,1",morvan
3,14,14,Calvados,38.0,297.0,5181,372938.0,"0,7","0,1",NaN
4,56,56,Morbihan,51.0,267.0,5399,367875.0,"0,9","0,1",bretagne


### 3.2 - Des donnés null ?
> exploration des données nulles 

In [6]:
data.isnull().sum()

dep-code                  0
dep-code-txt              0
dep-nom                   0
sapin-nb-exploitation    17
sapin-nb-hectare         19
total-nb-expl             0
total-nb-ha               2
pct-expl                  0
pct-hect                  0
region                   93
dtype: int64

### 3.3 - Compléter les données régions

In [7]:
url_dep = "https://www.data.gouv.fr/fr/datasets/r/987227fb-dcb2-429e-96af-8979f97c9c84"
dep = pd.read_csv(url_dep)
dep.head()

,num_dep,dep_name,region_name
0,01,Ain,Auvergne-Rhône-Alpes
1,02,Aisne,Hauts-de-France
2,03,Allier,Auvergne-Rhône-Alpes
3,04,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur
4,05,Hautes-Alpes,Provence-Alpes-Côte d'Azur


In [8]:
# Test de la connexion entre les deux éléments
dep[dep["dep_name"] == "Nièvre"]

,num_dep,dep_name,region_name
58,58,Nièvre,Bourgogne-Franche-Comté


In [9]:
dep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   num_dep      101 non-null    object
 1   dep_name     101 non-null    object
 2   region_name  101 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [10]:
dep = dep[["num_dep", "region_name"]]
dep.head()

,num_dep,region_name
0,01,Auvergne-Rhône-Alpes
1,02,Hauts-de-France
2,03,Auvergne-Rhône-Alpes
3,04,Provence-Alpes-Côte d'Azur
4,05,Provence-Alpes-Côte d'Azur


In [11]:
data = data.merge(dep, left_on="dep-code-txt", right_on="num_dep", how="inner")
data = data.drop(["num_dep", "dep-code", "region", "pct-expl", "pct-hect"], axis=1)
data.head()

,dep-code-txt,dep-nom,sapin-nb-exploitation,sapin-nb-hectare,total-nb-expl,total-nb-ha,region_name
0,58,Nièvre,59.0,879.0,2715,367818.0,Bourgogne-Franche-Comté
1,29,Finistère,47.0,650.0,5978,382582.0,Bretagne
2,21,Côte-d'Or,39.0,378.0,4093,462011.0,Bourgogne-Franche-Comté
3,14,Calvados,38.0,297.0,5181,372938.0,Normandie
4,56,Morbihan,51.0,267.0,5399,367875.0,Bretagne


In [12]:
data.isnull().sum()

dep-code-txt              0
dep-nom                   0
sapin-nb-exploitation    17
sapin-nb-hectare         19
total-nb-expl             0
total-nb-ha               2
region_name               0
dtype: int64

### 3.4 - Remplacer les lignes vides par des 0

In [13]:
# Création des filtres
mask_filter_1 = data["sapin-nb-exploitation"].isnull()
mask_filter_2 = data["sapin-nb-hectare"].isnull()
mask_filter_3 = data["total-nb-ha"].isnull()

In [14]:
# Marquer les données qui seront remplacer pour en garder l'historique
data.loc[mask_filter_1, "sapin-nb-exploitation-is-null"] = "True"
data.loc[~mask_filter_1, "sapin-nb-exploitation-is-null"] = "False"

data.loc[mask_filter_2, "sapin-nb-hectare-is-null"] = "True"
data.loc[~mask_filter_2, "sapin-nb-hectare-is-null"] = "False"

data.loc[mask_filter_3, "total-nb-ha-is-null"] = "True"
data.loc[~mask_filter_3, "total-nb-ha-is-null"] = "False"

In [15]:
# Remplacer les valeurs
values = {"sapin-nb-exploitation": 0.0, "sapin-nb-hectare": 0.0, "total-nb-ha": 0.0}
data = data.fillna(value=values)

In [16]:
data[data["sapin-nb-exploitation-is-null"] == "True"].head()

,dep-code-txt,dep-nom,sapin-nb-exploitation,sapin-nb-hectare,total-nb-expl,total-nb-ha,region_name,sapin-nb-exploitation-is-null,sapin-nb-hectare-is-null,total-nb-ha-is-null
81,66,Pyrénées-Orientales,0.0,3.0,3179,68021.0,Occitanie,True,False,False
82,06,Alpes-Maritimes,0.0,0.0,1154,41141.0,Provence-Alpes-Côte d'Azur,True,True,False
85,2A,Corse-du-Sud,0.0,0.0,1003,63011.0,Corse,True,True,False
86,91,Essonne,0.0,0.0,653,83078.0,Île-de-France,True,True,False
87,971,Guadeloupe,0.0,0.0,7232,31836.0,Guadeloupe,True,True,False


In [17]:
data[data["sapin-nb-hectare-is-null"] == "True"].head()

,dep-code-txt,dep-nom,sapin-nb-exploitation,sapin-nb-hectare,total-nb-expl,total-nb-ha,region_name,sapin-nb-exploitation-is-null,sapin-nb-hectare-is-null,total-nb-ha-is-null
82,06,Alpes-Maritimes,0.0,0.0,1154,41141.0,Provence-Alpes-Côte d'Azur,True,True,False
83,07,Ardèche,9.0,0.0,3661,122127.0,Auvergne-Rhône-Alpes,False,True,False
84,13,Bouches-du-Rhône,15.0,0.0,3793,145713.0,Provence-Alpes-Côte d'Azur,False,True,False
85,2A,Corse-du-Sud,0.0,0.0,1003,63011.0,Corse,True,True,False
86,91,Essonne,0.0,0.0,653,83078.0,Île-de-France,True,True,False


In [18]:
data[data["total-nb-ha-is-null"] == "True"].head()

,dep-code-txt,dep-nom,sapin-nb-exploitation,sapin-nb-hectare,total-nb-expl,total-nb-ha,region_name,sapin-nb-exploitation-is-null,sapin-nb-hectare-is-null,total-nb-ha-is-null
90,92,Hauts-de-Seine,0.0,0.0,5,0.0,Île-de-France,True,True,True
99,75,Ville de Paris,0.0,0.0,5,0.0,Île-de-France,True,True,True


In [19]:
data.isnull().sum()

dep-code-txt                     0
dep-nom                          0
sapin-nb-exploitation            0
sapin-nb-hectare                 0
total-nb-expl                    0
total-nb-ha                      0
region_name                      0
sapin-nb-exploitation-is-null    0
sapin-nb-hectare-is-null         0
total-nb-ha-is-null              0
dtype: int64

## 3 - Export des données propres

In [20]:
data.to_csv("Documents/geoDataScience/defi_carto_sapin_2024/data/cleaned_sapin_data.csv", index=False)

-- END -- 